# ACTUS ANN Contract: 30-Year Fixed-Rate Mortgage

This notebook demonstrates the **Annuity (ANN)** contract type from the ACTUS standard.

## What is an ANN Contract?

An Annuity contract is characterized by:
- Fixed periodic payments (principal + interest)
- Gradually increasing principal repayment
- Gradually decreasing interest component
- Common use: Fixed-rate mortgages

## Example: $500,000 Mortgage at 6% for 30 Years

We'll simulate a typical home mortgage with:
- Principal: $500,000
- Interest Rate: 6% per annum
- Term: 30 years
- Monthly payments

In [ ]:
import jax.numpy as jnp
from jactus.contracts import create_contract
from jactus.core import (
    ContractAttributes,
    ContractType,
    ContractRole,
    ActusDateTime,
)
from jactus.observers import ConstantRiskFactorObserver

## Step 1: Define Contract Attributes

In [ ]:
# Contract dates
status_date = ActusDateTime(2024, 1, 1, 0, 0, 0)
initial_exchange = ActusDateTime(2024, 1, 15, 0, 0, 0)
maturity_date = ActusDateTime(2054, 1, 15, 0, 0, 0)  # 30 years

# Create ANN contract attributes
attrs = ContractAttributes(
    contract_id="MORTGAGE-001",
    contract_type=ContractType.ANN,
    contract_role=ContractRole.RPA,  # Real Position Asset (lender perspective)
    status_date=status_date,
    initial_exchange_date=initial_exchange,
    maturity_date=maturity_date,
    notional_principal=500_000.0,
    nominal_interest_rate=0.06,  # 6% annual
    day_count_convention="A360",
    interest_payment_cycle="1M",  # Monthly payments
    cycle_of_principal_redemption="1M",  # Monthly principal repayment
    currency="USD",
)

print(f"Contract ID: {attrs.contract_id}")
print(f"Principal: ${attrs.notional_principal:,.2f}")
print(f"Interest Rate: {attrs.nominal_interest_rate*100}%")
print(f"Term: {(maturity_date.year - initial_exchange.year)} years")

## Step 2: Create Risk Factor Observer

In [ ]:
# Create risk factor observer (constant rate)
rf_observer = ConstantRiskFactorObserver(constant_value=0.06)

## Step 3: Create and Simulate Contract

In [ ]:
# Create contract
mortgage = create_contract(attrs, rf_observer)

# Simulate contract
result = mortgage.simulate(rf_observer)

print(f"Total events generated: {len(result.events)}")
print(f"Expected payments: {12 * 30} (monthly for 30 years)")

## Step 4: Analyze Payment Schedule

In [ ]:
# Extract events
events = result.events

# Show first few payments
print("\nFirst 5 Monthly Payments:")
print("=" * 80)
print(f"{'Date':<12} {'Event':<6} {'Payment':<12} {'Principal':<12} {'Interest':<12}")
print("=" * 80)

for i, event in enumerate(events[:5]):
    payment = abs(float(event.payoff))
    date_str = f"{event.event_time.year}-{event.event_time.month:02d}-{event.event_time.day:02d}"
    
    # Get principal from state change
    principal_change = float(event.state_pre.nt - event.state_post.nt)
    interest = payment - principal_change if principal_change > 0 else payment
    
    print(f"{date_str:<12} {event.event_type.value:<6} ${payment:>10.2f} ${principal_change:>10.2f} ${interest:>10.2f}")

## Step 5: Calculate Total Interest Paid

In [ ]:
# Calculate totals
total_payments = sum(abs(float(e.payoff)) for e in events if e.event_type.value in ['IP', 'PR'])
total_interest = total_payments - 500_000.0

print(f"\nMortgage Summary:")
print(f"=" * 50)
print(f"Principal borrowed:     ${500_000:>12,.2f}")
print(f"Total paid over 30 yrs: ${total_payments:>12,.2f}")
print(f"Total interest paid:    ${total_interest:>12,.2f}")
print(f"Interest as % of loan:  {(total_interest/500_000)*100:>11.1f}%")

## Step 6: Visualize Amortization Schedule

In [ ]:
import matplotlib.pyplot as plt

# Extract data for plotting
months = list(range(len(events)))
principal_balances = [float(e.state_post.nt) for e in events]
principal_payments = []
interest_payments = []

for event in events:
    payment = abs(float(event.payoff))
    principal_change = float(event.state_pre.nt - event.state_post.nt)
    interest = payment - principal_change if principal_change > 0 else payment
    principal_payments.append(principal_change)
    interest_payments.append(interest)

# Create figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Principal Balance Over Time
ax1.plot(months, principal_balances, linewidth=2, color='navy')
ax1.set_xlabel('Payment Number (Months)', fontsize=12)
ax1.set_ylabel('Outstanding Principal ($)', fontsize=12)
ax1.set_title('30-Year Mortgage: Principal Balance', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.ticklabel_format(style='plain', axis='y')

# Plot 2: Principal vs Interest Components
ax2.fill_between(months, 0, interest_payments, label='Interest', alpha=0.7, color='red')
ax2.fill_between(months, interest_payments, 
                 [p+i for p,i in zip(principal_payments, interest_payments)], 
                 label='Principal', alpha=0.7, color='green')
ax2.set_xlabel('Payment Number (Months)', fontsize=12)
ax2.set_ylabel('Payment Amount ($)', fontsize=12)
ax2.set_title('30-Year Mortgage: Payment Composition', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Charts show:")
print("  • Top: Principal balance decreases over time")
print("  • Bottom: Early payments are mostly interest, later payments mostly principal")

## Key Observations

The ANN contract type correctly implements:

1. **Constant Payment Amount**: Each month the total payment is the same
2. **Changing Composition**: Early payments are mostly interest, later payments mostly principal
3. **Amortization Schedule**: Principal balance decreases steadily to zero at maturity
4. **ACTUS Compliance**: All calculations follow the ACTUS v1.1 specification

## References

- ACTUS Specification v1.1, Section 7.5 - Annuity (ANN)
- [ACTUS Technical Specification](https://www.actusfrf.org/)